# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770909699349                   -0.52    9.0         
  2   -2.772149429487       -2.91       -1.32    1.0    155ms
  3   -2.772170332472       -4.68       -2.47    1.0    139ms
  4   -2.772170653902       -6.49       -3.19    1.0    140ms
  5   -2.772170722792       -7.16       -4.18    2.0    178ms
  6   -2.772170722871      -10.10       -4.34    1.0    143ms
  7   -2.772170723009       -9.86       -5.12    1.0    161ms
  8   -2.772170723013      -11.38       -5.26    1.0    149ms
  9   -2.772170723015      -11.75       -6.13    1.0    174ms
 10   -2.772170723015      -13.22       -6.14    2.0    170ms
 11   -2.772170723015      -13.25       -6.78    1.0    165ms
 12   -2.772170723015      -14.01       -7.05    1.0    155ms
 13   -2.772170723015   +    -Inf       -7.29    2.0    179ms
 14   -2.772170723015      -13.85       -8.09    1.0    170ms
n     Ene

1.7735580375705744

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770763969594                   -0.52    8.0         
  2   -2.772057440669       -2.89       -1.32    1.0    137ms
  3   -2.772082856241       -4.59       -2.47    1.0    166ms
  4   -2.772083320546       -6.33       -3.17    1.0    141ms
  5   -2.772083414947       -7.03       -3.77    2.0    202ms
  6   -2.772083417748       -8.55       -5.25    1.0    143ms
  7   -2.772083417810      -10.21       -5.57    3.0    180ms
  8   -2.772083417810      -12.44       -5.98    1.0    154ms
  9   -2.772083417811      -12.69       -7.22    1.0    151ms
 10   -2.772083417811      -13.94       -8.04    2.0    194ms

Polarizability via ForwardDiff:       1.77253497488744
Polarizability via finite difference: 1.7735580375705744
